In [1]:
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path="employees.csv")
documents = loader.load()

### Chunking
#### Splitting characters using various chunk sizes and chunk overlaps

In [2]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
for text in texts:
    print(text)

page_content='employee_id: b2b4e014-fa01-40b9-aeb7-4182a72c31da\nname: Brianna\ndesignation: Software Engineer' metadata={'source': 'employees.csv', 'row': 0}
page_content='employee_id: eb6d79c3-a8e1-4d3b-8027-33dae18990e0\nname: Jennifer\ndesignation: Senior Software Engineer' metadata={'source': 'employees.csv', 'row': 1}
page_content='employee_id: a67a773d-89f9-4bb2-8a1c-c68db84b0028\nname: Jessica\ndesignation: Team Lead' metadata={'source': 'employees.csv', 'row': 2}
page_content='employee_id: 97b6548d-ad5c-4f89-8f0e-fcd90478861e\nname: Cheryl\ndesignation: Project Manager' metadata={'source': 'employees.csv', 'row': 3}
page_content='employee_id: 1e7a8ba4-0741-4707-b8dc-921ba3909760\nname: Abigail\ndesignation: Technical Architect' metadata={'source': 'employees.csv', 'row': 4}
page_content='employee_id: b0452d53-e3c8-415a-b841-6e696f43278e\nname: Maxwell\ndesignation: Business Analyst' metadata={'source': 'employees.csv', 'row': 5}
page_content='employee_id: 2f5a4aef-5950-4522-8d

### Word embeddings
#### Trying various vector embedding models

In [3]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

C:\Users\New\anaconda3\envs\SLMs\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\New\anaconda3\envs\SLMs\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\New\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Storing embedding in Vector DB

In [5]:
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=texts, embedding=embeddings)

In [6]:
from langchain_community.llms import Ollama
llm = Ollama(model = "phi3")

### Querying LLM using own stored vector DB

In [7]:
from langchain.chains import ConversationalRetrievalChain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    vectorstore.as_retriever(),
    return_source_documents=True
)

In [8]:
import sys
chat_history = []
while True:
    query = input('Prompt: ')
    if query == "exit" or query == "quit" or query == "q":
        print('Exiting')
        sys.exit()
    result = qa_chain({'question': query, 'chat_history': chat_history})
    print('Answer: ' + result['answer'] + '\n')
    chat_history.append((query, result['answer']))

Prompt:  How many employees are there?


C:\Users\New\anaconda3\envs\SLMs\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Answer:  There are four employees. Their employee IDs, names, and designations are as follows:
1. Employee ID: 1346136c-cb11-491d-8262-3b5533cc5195 - Name: Dakota - Designation: Product Manager
2. Employee ID: 7440481b-82a7-4b25-9671-98dc77521ebc - Name: John - Designation: Product Manager
3. Employee ID: c18c02cc-168d-4697-a346-166f621e0858 - Name: Felicia - Designation: Product Manager
4. Employee ID: 020dec9b-dcf8-43dc-aec5-0a152dc9b681 - Name: Steven - Designation: Product Manager



Prompt:  exit


Exiting


SystemExit: 

C:\Users\New\anaconda3\envs\SLMs\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
